# SDAE Application: ISOLET Protein Expression

In [1]:
# import libraries

import warnings
warnings.filterwarnings('ignore')

# Import the numpy and pandas package
import numpy as np
import pandas as pd
import random
from sklearn.ensemble import ExtraTreesClassifier
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error,accuracy_score
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import optimizers,initializers,regularizers
from tensorflow.keras.callbacks import LambdaCallback
from sklearn.linear_model import LinearRegression


## Load DataSets

In [2]:
isolet_names = pd.read_csv('data\isolet.names',delimiter=':',header=None,skiprows=1).drop(columns=1).values.flatten()
isolet_cols = np.append(isolet_names,['target'])
isolet_data = pd.read_csv('data\isolet1+2+3+4.data',names=isolet_cols,index_col=False)

isolet_test_data = pd.read_csv('data\isolet5.data',names=isolet_cols,index_col=False)

In [3]:
isolet_data

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f609,f610,f611,f612,f613,f614,f615,f616,f617,target
0,-0.4394,-0.0930,0.1718,0.4620,0.6226,0.4704,0.3578,0.0478,-0.1184,-0.2310,...,0.4102,0.2052,0.3846,0.3590,0.5898,0.3334,0.6410,0.5898,-0.4872,1.0
1,-0.4348,-0.1198,0.2474,0.4036,0.5026,0.6328,0.4948,0.0338,-0.0520,-0.1302,...,0.0000,0.2954,0.2046,0.4772,0.0454,0.2046,0.4318,0.4546,-0.0910,1.0
2,-0.2330,0.2124,0.5014,0.5222,-0.3422,-0.5840,-0.7168,-0.6342,-0.8614,-0.8318,...,-0.1112,-0.0476,-0.1746,0.0318,-0.0476,0.1112,0.2540,0.1588,-0.4762,2.0
3,-0.3808,-0.0096,0.2602,0.2554,-0.4290,-0.6746,-0.6868,-0.6650,-0.8410,-0.9614,...,-0.0504,-0.0360,-0.1224,0.1366,0.2950,0.0792,-0.0072,0.0936,-0.1510,2.0
4,-0.3412,0.0946,0.6082,0.6216,-0.1622,-0.3784,-0.4324,-0.4358,-0.4966,-0.5406,...,0.1562,0.3124,0.2500,-0.0938,0.1562,0.3124,0.3124,0.2188,-0.2500,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6233,-0.5742,0.1050,0.4936,0.3986,-0.2058,-0.4130,-0.4188,-0.5194,-0.5080,-0.4878,...,0.5000,0.6800,0.8200,0.8400,0.8400,0.7400,0.8200,0.6400,0.3200,22.0
6234,-0.4520,0.0154,0.5078,0.8978,0.7956,0.4366,0.2352,0.1300,0.0682,0.3004,...,0.5000,0.2250,0.7500,0.8750,0.6750,0.6000,0.4500,-0.1250,-0.2250,23.0
6235,-0.5824,-0.1646,0.1406,0.6224,0.6626,0.3172,0.0924,0.0120,-0.1646,-0.1326,...,0.8068,0.7392,0.7392,0.6908,0.7294,0.7004,0.6812,0.5170,0.3430,24.0
6236,0.0160,0.8168,1.0000,0.7814,0.4084,0.2122,-0.2218,-0.6848,-0.8424,-0.7588,...,0.0344,0.0344,-0.0344,0.4252,0.2874,-0.0114,0.1034,-0.1954,-0.8620,25.0


In [4]:
isolet_data.describe()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f609,f610,f611,f612,f613,f614,f615,f616,f617,target
count,6238.000000,6238.000000,6238.000000,6238.000000,6238.000000,6238.000000,6238.000000,6238.000000,6238.000000,6238.000000,...,6238.000000,6238.000000,6238.000000,6238.000000,6238.000000,6238.000000,6238.000000,6238.000000,6238.000000,6238.000000
mean,-0.394077,0.141918,0.343198,0.445395,0.321922,0.162902,0.024053,-0.082475,-0.149804,-0.192733,...,0.332786,0.303167,0.312427,0.326087,0.293630,0.220475,0.180992,0.082916,-0.272561,13.502405
std,0.240579,0.322356,0.325086,0.308172,0.465508,0.572196,0.559857,0.535643,0.528610,0.514014,...,0.336196,0.333184,0.337526,0.338478,0.338831,0.339323,0.341432,0.336243,0.357741,7.500601
min,-1.000000,-0.892600,-0.975200,-0.968000,-0.996600,-0.984800,-1.000000,-1.000000,-1.000000,-1.000000,...,-0.729800,-0.775200,-0.830200,-0.711800,-0.770800,-0.810600,-0.780200,-0.802800,-0.962600,1.000000
25%,-0.555200,-0.100800,0.099050,0.247200,-0.037900,-0.365950,-0.461550,-0.534400,-0.579550,-0.604150,...,0.083400,0.057450,0.064300,0.077000,0.041600,-0.025600,-0.066600,-0.156700,-0.541350,7.000000
50%,-0.423000,0.106600,0.330000,0.449000,0.419000,0.208100,-0.003000,-0.151000,-0.225600,-0.255200,...,0.326700,0.283000,0.289000,0.304400,0.274000,0.186600,0.147800,0.046800,-0.318200,14.000000
75%,-0.248450,0.355550,0.578750,0.661950,0.695600,0.685350,0.491200,0.303050,0.187900,0.118450,...,0.583950,0.544600,0.561400,0.574700,0.540150,0.458000,0.407200,0.296100,-0.054000,20.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.889800,26.000000


In [5]:
isolet_data.min()

f1       -1.0000
f2       -0.8926
f3       -0.9752
f4       -0.9680
f5       -0.9966
           ...  
f614     -0.8106
f615     -0.7802
f616     -0.8028
f617     -0.9626
target    1.0000
Length: 618, dtype: float64

In [6]:
isolet_data.max()

f1         1.0000
f2         1.0000
f3         1.0000
f4         1.0000
f5         1.0000
           ...   
f614       1.0000
f615       1.0000
f616       1.0000
f617       0.8898
target    26.0000
Length: 618, dtype: float64

In [7]:
print(isolet_data.isna().sum().any())
print(isolet_test_data.isna().sum().any())

False
False


In [8]:

X_train = isolet_data.iloc[:,:-1]
y_train = isolet_data.iloc[:,-1]
X_test = isolet_test_data.iloc[:,:-1]
y_test = isolet_test_data.iloc[:,-1]


In [9]:
X_train.shape

(6238, 617)

In [10]:
y_train.shape

(6238,)

In [11]:
y_test.shape

(1559,)

## SDAE functions

In [12]:
# Weighted layer defintion with zero-to-one clipping constraint and 1s initialzation with Lasso regularization
class SelectiveLayer(keras.layers.Layer):
    def __init__(self, lasso_rate=0.001,convergence_rate=0.001,*args, **kwargs):
        super().__init__(*args, **kwargs)
        self.lasso_rate = lasso_rate
        #self.convergence_rate = convergence_rate
        
    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", shape=(int(input_shape[-1]),), 
                                      initializer = initializers.RandomUniform(minval=0.999999, maxval=0.9999999),
                                      regularizer=regularizers.l1(self.lasso_rate),
                                      constraint = ZeroToOneClip()
                                     )
    def call(self, inputs):
        return tf.multiply(inputs, self.kernel)
    def get_config(self):
        config = super().get_config().copy()
        return config


class ZeroToOneClip(tf.keras.constraints.Constraint):
    def __call__(self, w):
        w_new = tf.clip_by_value(w, 0, 1)
        return w_new



def SelectDAE(input_shape,
                  nbr_hidden_layers =1,
                  hidden_layer_shape=13,
                  encodings_nbr = 6,
                  activation="linear",
                  lasso_rate=0.001,
                  sl_lasso_rate=0.1):
    #Encoder 1:Input
    feature_inputs = Input(shape=[input_shape],name='input')
    #Encoder 2: Input
    selective_layer = SelectiveLayer(lasso_rate=sl_lasso_rate)
    feature_selection_choose = selective_layer(feature_inputs)
    
    for i in range(nbr_hidden_layers):
        #Encoder 1: hidden layers
        encoder_layer_full = Dense(hidden_layer_shape, activation=activation,
                            name='encoder_hidden_layer_full_'+str(i))
        #Encoder 2: hidden layers
        encoder_layer_select = Dense(hidden_layer_shape, activation=activation,
                            kernel_regularizer=regularizers.l1(lasso_rate),
                            bias_regularizer=regularizers.l1(lasso_rate),
                            name='encoder_hidden_layer_select_'+str(i))
        if i==0:
            encoder_layer_full_output = encoder_layer_full(feature_inputs)
            encoder_layer_select_output = encoder_layer_select(feature_selection_choose)
        else:
            encoder_layer_full_output = encoder_layer_full(encoder_layer_full_output)
            encoder_layer_select_output = encoder_layer_select(encoder_layer_select_output)
    
    #Encoder 1: Encodings
    encoding_layer_full = Dense(encodings_nbr, activation=activation,
                           name='encoding_layer_full')
    encoding_layer_full_output = encoding_layer_full(encoder_layer_full_output)
    
    #Encoder 2: Encodings
    encoding_layer_select = Dense(encodings_nbr, activation=activation,
                           kernel_regularizer=regularizers.l1(lasso_rate),
                           bias_regularizer=regularizers.l1(lasso_rate),
                           name='encoding_layer_select')
    encoding_layer_select_output = encoding_layer_select(encoder_layer_select_output)
    
    #Decoder Layers
    for i in range(nbr_hidden_layers):
        decoder_layer = Dense(hidden_layer_shape, activation=activation,
                            name='decoder_hidden_layer_'+str(i))
        if i==0:
            decoder_layer_full_output = decoder_layer(encoding_layer_full_output)
            decoder_layer_select_output = decoder_layer(encoding_layer_select_output)
        else:
            decoder_layer_full_output = decoder_layer(decoder_layer_full_output)
            decoder_layer_select_output = decoder_layer(decoder_layer_select_output)
    
    #Reconstruction Layer
    reconstruction_layer = Dense(input_shape, activation='linear',
                                 name='reconstruction_layer')
    recons_layer_full_output = reconstruction_layer(decoder_layer_full_output)
    recons_layer_select_output = reconstruction_layer(decoder_layer_select_output)

    latent_encoder_full = Model(feature_inputs, encoding_layer_full_output,name='fullFeats_Encoder')
    latent_encoder_select = Model(feature_inputs, encoding_layer_select_output,name='SelectFeats_Encoder')
    feature_selection_output=Model(feature_inputs,feature_selection_choose,name='SelectFeats_Layer')
    autoencoder = Model(feature_inputs, recons_layer_full_output,name='DAE')
    autoencoder_select = Model(feature_inputs, recons_layer_select_output,name='SelectDAE')   
    
    print('Autoencoder Structure-------------------------------------')
    autoencoder.summary()
    
    print('SelectEncoder Structure-------------------------------------')
    latent_encoder_select.summary()
    
    return autoencoder,autoencoder_select,feature_selection_output, latent_encoder_full,latent_encoder_select


def top_k_keep(p_arr_,p_top_k_):
    top_k_idx=p_arr_.argsort()[::-1][0:p_top_k_]
    top_k_value=p_arr_[top_k_idx]
    #print(str(top_k_value))
    return np.where(p_arr_<top_k_value[-1],0,p_arr_)

def show_feature_selection(autoencoder,p_key_number=36):
    data = autoencoder.layers[1].get_weights()[0]
    weight_top_k=top_k_keep(np.array(data),p_key_number)
    eliminated_feats = np.where(weight_top_k==0)[0]
    survived_feats = np.where(weight_top_k!=0)[0]
    return weight_top_k,eliminated_feats,survived_feats

## Build Model

In [13]:
#For Repeatability: select a run from 0->4
run = 0 #0,1,2,3,4
seed_list = [1,10,100,1000,10000]
lasso_rate_SL_list = [8e-3,9.25e-3,8.25e-3,9e-3,8.5e-3]

#parameters
seed = seed_list[run]
lasso_rate = 1e-5
hidden_layer_shape = 300
nbr_hidden_layers =1
encodings_nbr = 300
lasso_rate_SL = lasso_rate_SL_list[run]
activation = "selu"
learning_rate = 1e-3
nbr_batches = 20
batch_size = int(np.floor(X_train.shape[0]/nbr_batches))

#Set seed
random.seed(seed)
rndm_seed = random.randint(1,10000)
tf.random.set_seed(rndm_seed)

DAE,\
Select_DAE,\
SelectLayer_output,\
FullFeats_Encoder,\
SelectFeats_Encoder=SelectDAE(input_shape=X_train.shape[1],
                              nbr_hidden_layers = nbr_hidden_layers,
                              hidden_layer_shape=hidden_layer_shape,
                              encodings_nbr =encodings_nbr,
                              activation=activation,
                              lasso_rate=lasso_rate,
                              sl_lasso_rate=lasso_rate_SL)

Autoencoder Structure-------------------------------------
Model: "DAE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 617)]             0         
                                                                 
 encoder_hidden_layer_full_0  (None, 300)              185400    
  (Dense)                                                        
                                                                 
 encoding_layer_full (Dense)  (None, 300)              90300     
                                                                 
 decoder_hidden_layer_0 (Den  (None, 300)              90300     
 se)                                                             
                                                                 
 reconstruction_layer (Dense  (None, 617)              185717    
 )                                                               
    

In [14]:
#Pre-training
DAE.compile(loss='mean_squared_error',optimizer=optimizers.Adam())
DAE.fit(X_train,X_train,epochs=400,batch_size=batch_size,\
                        shuffle=True
                       )

Epoch 1/400
21/21 [==============================] - 1s 11ms/step - loss: 0.3024
Epoch 2/400
21/21 [==============================] - 0s 10ms/step - loss: 0.1157
Epoch 3/400
21/21 [==============================] - 0s 11ms/step - loss: 0.0831
Epoch 4/400
21/21 [==============================] - 0s 11ms/step - loss: 0.0691
Epoch 5/400
21/21 [==============================] - 0s 11ms/step - loss: 0.0603
Epoch 6/400
21/21 [==============================] - 0s 11ms/step - loss: 0.0541
Epoch 7/400
21/21 [==============================] - 0s 11ms/step - loss: 0.0494
Epoch 8/400
21/21 [==============================] - 0s 11ms/step - loss: 0.0457
Epoch 9/400
21/21 [==============================] - 0s 11ms/step - loss: 0.0427
Epoch 10/400
21/21 [==============================] - 0s 11ms/step - loss: 0.0401
Epoch 11/400
21/21 [==============================] - 0s 11ms/step - loss: 0.0377
Epoch 12/400
21/21 [==============================] - 0s 11ms/step - loss: 0.0358
Epoch 13/400
21/21 [=====

In [15]:
train_prediction_all_feats = DAE.predict(X_train)
print('Training:')
print('MSE Full Scaled Feats: '+str(mean_squared_error(X_train,train_prediction_all_feats)))

195/195 [==============================] - 1s 6ms/step
Training:
MSE Full Scaled Feats: 0.005619112066465677


In [16]:
test_prediction_all_feats = DAE.predict(X_test)
print("Testing:")
print('MSE Full Scaled Feats: '+str(mean_squared_error(X_test,test_prediction_all_feats)))

49/49 [==============================] - 0s 6ms/step
Testing:
MSE Full Scaled Feats: 0.006968829831172031


In [17]:
#Transfer Learning
for lay in range(nbr_hidden_layers+1):
    SelectFeats_Encoder.layers[-(1+lay)].set_weights(FullFeats_Encoder.layers[-(1+lay)].get_weights())

print_SL_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(' - surv feats: '+str(len(np.where(SelectLayer_output.layers[1].get_weights()[0]>1e-6)[0]))))


In [18]:
# Feature Selection Training
SelectFeats_Encoder.compile(loss='mean_squared_error',optimizer=optimizers.Adam(),metrics='mse')
DAE_encodings_train_scal = FullFeats_Encoder.predict(X_train)
SelectFeats_Encoder.fit(X_train,DAE_encodings_train_scal,epochs=350,batch_size=batch_size,\
                        shuffle=True,callbacks=[print_SL_weights])

195/195 [==============================] - 1s 3ms/step
Epoch 1/350
21/21 [==============================] - 1s 9ms/step - loss: 5.0214 - mse: 0.0221
Epoch 2/350
21/21 [==============================] - 0s 8ms/step - loss: 4.8983 - mse: 0.0039
Epoch 3/350
21/21 [==============================] - 0s 8ms/step - loss: 4.7919 - mse: 0.0026
Epoch 4/350
21/21 [==============================] - 0s 8ms/step - loss: 4.6870 - mse: 0.0029
Epoch 5/350
21/21 [==============================] - 0s 9ms/step - loss: 4.5825 - mse: 0.0036
Epoch 6/350
21/21 [==============================] - 0s 8ms/step - loss: 4.4782 - mse: 0.0043
Epoch 7/350
21/21 [==============================] - 0s 8ms/step - loss: 4.3740 - mse: 0.0052
Epoch 8/350
21/21 [==============================] - 0s 9ms/step - loss: 4.2700 - mse: 0.0060
Epoch 9/350
21/21 [==============================] - 0s 8ms/step - loss: 4.1661 - mse: 0.0068
Epoch 10/350
21/21 [==============================] - 0s 8ms/step - loss: 4.0623 - mse: 0.0077
Epoc

In [19]:
train_prediction_select_feats_scal = Select_DAE.predict(X_train)
print('Training:')
print('MSE Select Feats: '+str(mean_squared_error(X_train,train_prediction_select_feats_scal)))

195/195 [==============================] - 1s 6ms/step
Training:
MSE Select Feats: 0.056625742954059075


In [20]:
survived_feats = len(np.where(SelectLayer_output.layers[1].get_weights()[0]>=1e-6)[0])
print('Number of Important Features = '+str(survived_feats))

Number of Important Features = 49


In [21]:
wghts_final = SelectLayer_output.layers[1].get_weights()[0]
print(wghts_final)

[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 9.2437887e-08 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 4.6984330e-02 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 4.6489280e-02
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 3.9132025e-02 0.0000000e+00 5.1940434e-02 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 1.075

In [22]:
test_prediction_select_feats = Select_DAE.predict(X_test)
print('Testing:')
print('MSE Select Feats: '+str(mean_squared_error(X_test,test_prediction_select_feats)))

 1/49 [..............................] - ETA: 1s

49/49 [==============================] - 0s 6ms/step
Testing:
MSE Select Feats: 0.056948923388293285


In [23]:
_,_,top_k_feats = show_feature_selection(SelectLayer_output,p_key_number=survived_feats)
print(top_k_feats)

[ 31  39  45  47 107 134 142 144 159 178 186 193 195 198 204 208 211 213
 215 220 223 232 371 393 395 412 414 420 423 425 426 427 428 431 432 438
 439 440 441 455 463 472 476 479 513 576 577 579 584]


In [24]:
X_train_mask = X_train.values[:,top_k_feats]
X_test_mask = X_test.values[:,top_k_feats]
LR = LinearRegression(n_jobs = -1)
LR.fit(X_train_mask,X_train)
MSELR = mean_squared_error(X_test,LR.predict(X_test_mask))

In [25]:
print('Linear Reconstruction Error = '+str(MSELR/0.05327*0.015))

Linear Reconstruciton Error = 0.015111760717381385


In [26]:
X_train_mask = X_train.values[:,top_k_feats]
X_test_mask = X_test.values[:,top_k_feats]


In [27]:
clf = ExtraTreesClassifier(n_estimators=50, random_state=0)
clf.fit(X_train_mask,y_train)
y_pred = clf.predict(X_test_mask)

In [28]:
print('Accuracy Score = '+str(accuracy_score(y_test,y_pred)*100))

Accuracy Score = 87.49198203976908
